In [2]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import matplotlib.pyplot as plt
import os
import prepare
import acquire
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
telco_df = acquire.get_telco_data()
telco_df.head()

,Unnamed: 0,customer_id,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,...,signup_date,phone_service.1,multiple_lines.1,internet_service_type_id.1,online_security.1,online_backup.1,device_protection.1,tech_support.1,streaming_tv.1,streaming_movies.1
0,0,0002-ORFBO,2,1,2,Female,0,Yes,Yes,9,...,2021-04-21 18:07:34,Yes,No,1,No,Yes,No,Yes,Yes,No
1,1,0003-MKNFE,2,1,1,Male,0,No,No,9,...,2021-04-21 18:07:34,Yes,Yes,1,No,No,No,No,No,Yes
2,2,0004-TLHLJ,1,2,1,Male,0,No,No,4,...,2021-09-21 18:07:34,Yes,No,2,No,No,Yes,No,No,No
3,3,0011-IGKFF,1,2,1,Male,1,Yes,No,13,...,2020-12-21 18:07:34,Yes,No,2,No,Yes,Yes,No,Yes,Yes
4,4,0013-EXCHZ,2,2,1,Female,1,Yes,No,3,...,2021-10-21 18:07:34,Yes,No,2,No,No,No,Yes,Yes,No


In [5]:
telco_clean_df = prepare.clean_telco(telco_df)
telco_clean_df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,streaming_tv,streaming_movies,paperless_billing,monthly_charges,churn,contract_type,internet_service_type,payment_type,signup_date,total_charges
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,...,Yes,No,Yes,65.6,No,One year,DSL,Mailed check,2021-04-21 18:07:34,590.4
1,Male,0,No,No,9,Yes,Yes,No,No,No,...,No,Yes,No,59.9,No,Month-to-month,DSL,Mailed check,2021-04-21 18:07:34,539.1
2,Male,0,No,No,4,Yes,No,No,No,Yes,...,No,No,Yes,73.9,Yes,Month-to-month,Fiber optic,Electronic check,2021-09-21 18:07:34,295.6
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,...,Yes,Yes,Yes,98.0,Yes,Month-to-month,Fiber optic,Electronic check,2020-12-21 18:07:34,1274.0
4,Female,1,Yes,No,3,Yes,No,No,No,No,...,Yes,No,Yes,83.9,Yes,Month-to-month,Fiber optic,Mailed check,2021-10-21 18:07:34,251.7


In [12]:
telco_clean_df['charge_rank'] = pd.qcut(telco_clean_df['total_charges'], 4, labels = ['a','b','c','d'])
telco_clean_df

charges_corrected = telco_clean_df[telco_clean_df.total_charges != 0]
charges_corrected['total_charges'].describe()

count    7032.000000
mean     2283.147248
std      2264.703327
min        18.800000
25%       397.800000
50%      1394.575000
75%      3791.250000
max      8550.000000
Name: total_charges, dtype: float64

In [13]:
readable_telco = pd.get_dummies(telco_clean_df[['charge_rank', 'internet_service_type', 
                                                'contract_type']],drop_first = [True, True])
readable_telco

,charge_rank_b,charge_rank_c,charge_rank_d,internet_service_type_Fiber optic,internet_service_type_None,contract_type_One year,contract_type_Two year
0,1,0,0,0,0,1,0
1,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0
3,1,0,0,1,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
7038,0,0,0,1,0,0,0
7039,0,0,0,0,0,0,0
7040,1,0,0,0,0,1,0
7041,0,0,1,1,0,0,1
